# Asteroid Apparent Rates of Motion

Different types of Solar System objects are categorized in families according to their orbital characteristics and can be roughly identified in imaging data based on their rates of motion.  I've derived this information from the following sources:
Jedicke, R., 1996, ApJ, 111
Cortes-Contreras et al., 2019, MNRAS, https://arxiv.org/pdf/1909.11657.pdf (inc. moving object detection)
Savanevych, et al., 2018, A&A, https://www.aanda.org/articles/aa/pdf/2018/01/aa30323-16.pdf (inc. detection)

In [2]:
from astropy import units as u
from astropy import constants as c
import numpy as np
from astropy.coordinates import Angle

In [14]:
asteroids = {'Atira': {'orbital_radius': 0.7*c.au},
            'Vesta':  {'orbital_radius': 2.2*c.au},
            'MainBelt_B': {'orbital_radius':2.6*c.au},
            'MainBelt_C': {'orbital_radius':2.75*c.au},
            'MainBelt_D': {'orbital_radius':2.9*c.au},
            'MainBelt_E': {'orbital_radius':3.0*c.au},
            'MainBelt_F': {'orbital_radius':3.05*c.au},
            'MainBelt_G': {'orbital_radius':3.15*c.au},
            'Jupiter_Trojans': {'orbital_radius':5.203*c.au},
            'Transneptunian': {'orbital_radius':30.1*c.au},
            'KuiperBelt_inner': {'orbital_radius': 30.0*c.au},
            'KuiperBelt_outer': {'orbital_radius': 50.0*c.au}
            }

In [15]:
def calc_orb_period(a, object_mass = 0*u.kg):
    """Function to estimate orbital period assuming circular orbits"""
    P = np.sqrt( a**3 / ((c.G*(object_mass+c.M_sun)) / (4.0*np.pi**2)) )
    return P

In [16]:
def calc_orb_velocity(a,P):
    v = (2.0*np.pi*a)/P
    return v

In [17]:
def calc_apparent_angular_motion(params):
    Drel = abs(params['orbital_radius'] - 1.0*c.au)
    theta = np.sin( (params['orbital_velocity']*1.0*u.second/Drel).value )
    theta = Angle(theta, unit=u.rad)
    return theta

In [18]:
earth_orb_velocity = calc_orb_velocity(1.0*c.au,(365.24*u.d).to(u.s))
print('Earth mean orbital velocity = '+str(earth_orb_velocity))

Earth mean orbital velocity = 29786.069863739758 m / s


In [19]:
for family, params in asteroids.items():
    params['orbital_period'] = calc_orb_period(params['orbital_radius'])
    params['orbital_velocity'] = calc_orb_velocity(params['orbital_radius'],params['orbital_period'])
    params['min_rel_velocity'] = abs(earth_orb_velocity - params['orbital_velocity'])
    params['max_rel_velocity'] = abs(earth_orb_velocity + params['orbital_velocity'])
    params['app_ang_motion'] = calc_apparent_angular_motion(params)
    calc_apparent_angular_motion(params)
    asteroids[family] = params
    print(family+' P='+str(params['orbital_period'].to(u.d))+' v_orb='+str(params['orbital_velocity'])+' app ang motion = '+str(params['app_ang_motion'].to(u.arcsecond))+' per second')

Atira P=213.91709240562264 d v_orb=35599.515795038235 m / s app ang motion = 0.163615 arcsec per second
Vesta P=1191.8797691666737 d v_orb=20080.83513507824 m / s app ang motion = 0.0230728 arcsec per second
MainBelt_B P=1531.2935347404436 d v_orb=18471.681729576718 m / s app ang motion = 0.0159179 arcsec per second
MainBelt_C P=1665.7026155396502 d v_orb=17960.844962800442 m / s app ang motion = 0.014151 arcsec per second
MainBelt_D P=1803.8292451665125 d v_orb=17490.173274400684 m / s app ang motion = 0.0126923 arcsec per second
MainBelt_E P=1897.930517448549 d v_orb=17196.199845594023 m / s app ang motion = 0.011855 arcsec per second
MainBelt_F P=1945.575935714604 d v_orb=17054.66492972489 m / s app ang motion = 0.0114707 arcsec per second
MainBelt_G P=2042.040059798288 d v_orb=16781.772683752763 m / s app ang motion = 0.0107622 arcsec per second
Jupiter_Trojans P=4334.900822603986 d v_orb=13057.685314017654 m / s app ang motion = 0.00428358 arcsec per second
Transneptunian P=60318.

The question we are trying to answer is which of these objects move at a rate where their photometry crossing a microlensing event could be confused for an anomalous signature.  

Planetary anomalies last on the order of hours, and are typically observed every 15min.  So in order to be confused, a Solar System object would need to cross the stellar PSF of the source star (~2 arcsec in standard seeing conditions) in an interval between 15min to ~4hrs. 

From this, we can infer rates of motion that will be relevant for this study.

In [13]:
min_rate = 2.0*u.arcsecond / (4.0 * 3600.0)*u.second
max_rate = 2.0*u.arcsecond / (15.0 * 60.0)*u.second
print('Range of rates relevant to this study: ' + str(min_rate) + ' to ' + str(max_rate))

Range of rates relevant to this study: 0.0001388888888888889 arcsec s to 0.0022222222222222222 arcsec s


This implies that the likely false positives will come from the outer Solar System: Jupiter Trojans, Trans-Neptunian Objects and the Kuiper Belt.  